This jupyter notebook demonstrates how to set up a simple customisable Chatbot that can solve basic maths questions. The first thing we need to do is install to the jupyter notebook we are using the openai module. You do this you use the line "pip install openai", once the cell has run once it does not need to be run again so you can put a # in front of it to make it text instead of a piece of code so you do not accidentally run it again. 

In [ ]:
!pip install openai

Then we import the module

In [ ]:
from openai import OpenAI

We can then define the openai api key as a string as any time we make reference to a function that communicates with OpenAI we need to include it so it knows it is us using it. I've included the one here that I have been using but if you distribute it futher you'll most likely want to have someone else use their own or use a different one

In [ ]:
openai_api_key = ""

Next we define client within OpenAI using the api key string that we defined previously

In [ ]:
client = OpenAI(api_key=openai_api_key)

We can then using the extension .beta.assistants.create on the client to create the first assistant. We'll give it several properties to start off with. Name is how it is refered to as wtihin the OpenAI website. Instructions are the very basic prompts we give it as to how it should behave. We have given it a single tool here which is "code_interpreter" this allows the the chatbot to execute python code which allows it to perform data analysis, automate tasks and solve mathematical problems. The model decides which model to use.

In [ ]:
assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

Next we create a thread, a thread is a conversation between you and one or many assistants.

In [ ]:
thread = client.beta.threads.create()

Next we attach a message object to the thread, they can contain both texts and files and there is no limit on th number of messages able to be added to a file. Here thread_id decides the thread on which the message is added to. Role decides where the message is coming from, in this case the user is asking the assistant a question but we could have as easily had the message been from the system or the assistant. In the case of the system sending a message this would be essentialy identical to the instructions we gave to the assistant when creating the assistant. Usually in the case of the message being from the assistant these will be generated by the chatbot but can be manually entered as well. The content of the message is the string that the message contains and is what will actually be sent to be interpreted by the chatbot in this case. Feel free to try swapping out the content of the message to another simple maths question, something like "I need a basic y=x graph plot. Could you make it for me?" will show you the code for the plotted graph in python that the assistant makes.

In [ ]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

Next we define a class known as the EventHandler, it essentually decides on how the text from the chatbot is shown to us. The first import known as "override" is a decorator used to indicate a method is overriding a method in the superclass. The second import is the "AssistantEventHandler" which is the base class for handling events in OpenAI.

The class EventHandler has 4 functions defined inside of it which are used to define custom behaviours for different events in respons to the response stream that we get from the chat bot. The first function is the "on_text_created" which simply prints "assistant >" before the assistant responds so we know it is the assistant responding. The "on_text_delta" function is called dependent on a delta.value which gives a time dependence to the generation of text so that each piece of generated text by the assistant is given incramentally and provides real time feedback. "on_tool_call_created" is called when a tool is generated, in our case the code interpreter is our tool and so when it is used the code will print the type of tool initiated by the chatbot. The final function is "on_tool_call_delta" this adds like previous delta function incramental updates to the tool call such that when the code_interpreter tool is used it prints the code being sent to the code interpreter and provides detailed feedback on the codes execution. 

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler
  
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

Finally we retrieve the stream of responses for a specific thread, once again we define the thread_id that we have used and we do the same for the assistant_id, we can also provide some more simple instructions and finally we pass our event handler class into event_handler so the text comes out the way we want it from the chat bot. The stream.until_done() function gives our output.

In [ ]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Alex.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()